Impoting the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import librosa
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

Loading the Metadata into a Dataframe

In [2]:

metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


Function for extracting MFCC 

In [3]:
def MFCC(file):
    audio,sample_rate = librosa.load(file, res_type = 'kaiser_fast')
    mfcc_features = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
    mfcc_features_scaled = np.mean(mfcc_features.T, axis=0)
    return mfcc_features_scaled

Extracting MFCC for every Audio File

In [4]:
extracted_features = []
for i,row in metadata.iterrows():
    filename = os.path.join(os.path.abspath('UrbanSound8K/audio'), 'fold'+str(row["fold"])+'/', str(row["slice_file_name"]))
    label = row["class"]
    data = MFCC(filename)
    extracted_features.append([data, label])

c:\Users\Pankaj\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)
c:\Users\Pankaj\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
c:\Users\Pankaj\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)


Creating new Dataframe

In [5]:
df = pd.DataFrame(extracted_features, columns=['feature', 'class'])
df.head()

,feature,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.52066, 52.00812, -...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036354, ...",children_playing
4,"[-446.60352, 113.68541, -52.402206, 60.302044,...",children_playing


Splitting the Data into Testing Data and Training Data

In [6]:
x = np.array(df['feature'].to_list())
y = np.array(df['class'].to_list())
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)

Defining the Model Architecture

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import regularizers

In [8]:
models = Sequential([
    keras.layers.Dense(100, input_shape=(40,), activation = 'relu', kernel_regularizer = regularizers.L2(0.01)),
    keras.layers.Dense(200, activation = 'relu', kernel_regularizer = regularizers.L2(0.01)),
    keras.layers.Dense(100, activation = 'relu', kernel_regularizer = regularizers.L2(0.01)),
    keras.layers.Dense(10, activation = 'softmax', kernel_regularizer = regularizers.L2(0.01)),
])

Compiling the Model

In [9]:
models.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Fitting the Model

In [10]:
models.fit(x_train, y_train, batch_size = 32, validation_split = 0.1, epochs = 200)

Epoch 1/200
197/197 [==============================] - 9s 7ms/step - loss: 4.4348 - accuracy: 0.4605 - val_loss: 3.0949 - val_accuracy: 0.5565
Epoch 2/200
197/197 [==============================] - 1s 4ms/step - loss: 2.7782 - accuracy: 0.5947 - val_loss: 2.5936 - val_accuracy: 0.6123
Epoch 3/200
197/197 [==============================] - 1s 4ms/step - loss: 2.3626 - accuracy: 0.6508 - val_loss: 2.2323 - val_accuracy: 0.6724
Epoch 4/200
197/197 [==============================] - 1s 4ms/step - loss: 2.1668 - accuracy: 0.6712 - val_loss: 2.0774 - val_accuracy: 0.6767
Epoch 5/200
197/197 [==============================] - 1s 4ms/step - loss: 1.9417 - accuracy: 0.7011 - val_loss: 1.9676 - val_accuracy: 0.7024
Epoch 6/200
197/197 [==============================] - 1s 4ms/step - loss: 1.8303 - accuracy: 0.7222 - val_loss: 1.8019 - val_accuracy: 0.7096
Epoch 7/200
197/197 [==============================] - 1s 4ms/step - loss: 1.7258 - accuracy: 0.7289 - val_loss: 1.8017 - val_accuracy: 0.7024

Evaluating the Model

In [12]:
loss, accuracy = models.evaluate(x_test, y_test)
print(accuracy)

55/55 [==============================] - 0s 4ms/step - loss: 1.0116 - accuracy: 0.8271
0.8271322250366211


Building a Predictive System

In [14]:
input_audio = input('Enter the name of the image: ')
audio, sample_rate = librosa.load(input_audio, res_type = 'kaiser_fast')
mfcc_features = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
mfcc_features_scaled = np.mean(mfcc_features.T, axis=0)
mfcc_features_scaled = mfcc_features_scaled.reshape(1,-1)
prediction = models.predict(mfcc_features_scaled)
predictionLabel = labelencoder.inverse_transform([np.argmax(prediction)])
print(f'The Classified Sound: {predictionLabel[0].capitalize()}')

The Classified Sound: Siren
